In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from nltk.tag import pos_tag
import nltk
from nltk.corpus import stopwords
# nltk.download()

TAG_REMOVE = re.compile(r'<[^>]+>')

def clean_text(text): #removes punctuation, html tags, and pronouns that are not listed above
#     clean = TAG_REMOVE.sub('', text)
    clean = text.replace('<i>', '')
    clean = clean.replace('</i>','')
    clean = " ".join(re.compile('\w+').findall(clean))
    if 'LORD' in clean:
        clean = clean.replace('LORD','lord')
#     clean = re.sub(r'([A-Za-z]+eth\b)', r' ', clean)
#     clean = re.sub(r'([A-Za-z]+est\b)', r' ', clean)
    clean = " ".join(clean.split('eth '))
    clean = " ".join(clean.split('est '))
    for word in stopwords.words('english'):
        if word not in model:
            clean = clean.replace(' '+ word +' ', ' ')
#     tagged_sent = pos_tag(clean.split())
#     edited = [word for word,tag in tagged_sent if (tag != 'NNP' and tag != 'NNPS') or (tag == 'NNP' or tag == 'NNPS' and word in IMPORTANT_PRONOUNS_TO_KEEP)]
#     edited = ' '.join(edited)
    
    return clean

# bibles = pd.read_csv( 'bibles.csv' )
# bible_file = bibles[["Verse", "King James Bible"]]

# bible = {}
# #Example: bible['Matthew'][1][1] is Matthew 1:1
# for i in range(len(bible_file)):
#     book_chp_vrs = bible_file["Verse"][i].split()
#     book = " ".join(book_chp_vrs[:-1])
#     chap, verse = book_chp_vrs[-1].split(":")
#     chap = int(chap)
#     verse = int(verse)
#     text = bible_file["King James Bible"][i]
#     if book not in bible.keys():
#         if chap == 1 and verse == 1:
#             bible[book] = [[],['',clean_text(text)]] #insert clean text
#         else:
#             print("Error, not chapter 1")
#     else:
#         if len(bible[book]) == chap:
#             bible[book].append([clean_text(text)])
#         else:
            
#             bible[book][chap].append(clean_text(text)) #insert clean text 
            
# # for key in bible.keys():
# #     for chapter in bible[key]:
# #         for verse in chapter:
# #             print(verse)

# print("bible done")

In [7]:
import re

keep_words = ['twentieth','thirtieth','fortieth','fiftieth','sixtieth','seventieth','eightieth','ninetieth']
filter_words = {'twentieth':''}

def read_chapter(file, line, BoM):
    blank = next(file)
    line = next(file)
    while True: #Read each verse
        if line == 'THE END':
            break
        
        if chap_header_pattern.match(line):
            while not book_chp_vrs_pattern.match(line):
                line = next(file)
        book_chp_vrs = line.split()
#         print(book_chp_vrs)
        book = " ".join(book_chp_vrs[:-1])
        chap, verse = book_chp_vrs[-1].split(":")
        chap = int(chap)
        verse = int(verse)
        text = next(file).split()
        text = " ".join(text[1:])
        text = text.replace('\n','') + ' '
#         text += ' '
        line = text
        count = 0
        while True: #Read full text of verse
            count += 1
            line = next(file)
#             if book_chp_vrs_pattern.match(line) or line == 'THE END' or chap_header_pattern.match(line):
#                 break
            if line == '\n':
                while not book_chp_vrs_pattern.match(line) and line != 'THE END':
                    line = next(file)
                break
            text += line.replace('\n', ' ')
            if count > 100:
                print("100 error")
                break
        #Put verse in dicionary
        if book not in BoM.keys():
            if chap == 1 and verse == 1:
                BoM[book] = [[],['',clean_text(text)]] #insert clean text
            else:
                print("Error, not chapter 1")
        else:
            if len(BoM[book]) == chap:
                BoM[book].append([clean_text(text)])
            else:
                BoM[book][chap].append(clean_text(text)) #insert clean text
        
#         if pattern.match(line):
#             return line, file
            

pattern = re.compile("(Chapter \d+)")
book_chp_vrs_pattern = re.compile("(\d*( )*([A-Za-z]+ )*\d+:\d+)")
chap_header_pattern = re.compile("(\d*( )*([A-Za-z]+ )*\d+$)")

bom_file = open('book_of_mormon.txt')
BoM = {}
for line in bom_file:
    if pattern.match(line):
#         line, bom_file = read_chapter(bom_file, line)
        read_chapter(bom_file, line, BoM)
        
print("BoM done")

BoM done


In [160]:
for key in BoM.keys():
    for chapter in BoM[key]:
        for verse in chapter:
            print(verse)
            print()
print("End")



I Nephi having been born goodly parents therefore I was taught somewhat in all the learning my father having seen many afflictions in the course my days nevertheless having been highly favored the Lord in all my days yea having had great knowledge the goodness the mysteries God therefore I make record my proceedings in my days

Yea I make record in the language my father which consists the learning the Jews the language the Egyptians

And I know that the record which I make is true I make it with mine own hand I make it according my knowledge

For it came pass in the commencement the first year the reign Zedekiah king Judah my father Lehi having dwelt at Jerusalem in all his days in that same year there came many prophets prophesying unto the people that they must repent or the great city Jerusalem must be destroyed

Wherefore it came pass that my father Lehi as he went forth prayed unto the Lord yea even with all his heart in behalf his people

And it came pass as he prayed unto the


But the things which thou shalt see hereafter thou shalt not write for the Lord God hath ordained the apostle the Lamb God that he should write them

And also others who have been them hath he shown all things they have written them they are sealed up come forth in their purity according the truth which is in the Lamb in the own due time the Lord unto the house Israel

And I Nephi heard bear record that the name the apostle the Lamb was John according the word the angel

And behold I Nephi am forbidden that I should write the remainder the things which I saw heard wherefore the things which I have written suffic me I have written but small part the things which I saw

And I bear record that I saw the things which my father saw the angel the Lord did make them known unto me

And now I make an end speaking concerning the things which I saw while I was carried away in the spirit if all the things which I saw are not written the things which I have written are true And thus it is Amen

An

And it shall be as the chased roe as sheep that no man tak up they shall every man turn his own people flee every one into his own land

Every one that is proud shall be thrust through yea every one that is joined the wicked shall fall by the sword

Their children also shall be dashed pieces before their eyes their houses shall be spoiled their wives ravished

Behold I will stir up the Medes against them which shall not regard silver gold nor shall they delight in it

Their bows shall also dash the young men pieces they shall have no pity on the fruit the womb their eyes shall not spare children

And Babylon the glory kingdoms the beauty the Chaldees excellency shall be as when God overthrew Sodom Gomorrah

It shall never be inhabited neither shall it be dwelt in from generation generation neither shall the Arabian pitch tent there neither shall the shepherds make their fold there

But wild beasts the desert shall lie there their houses shall be full doleful creatures owls shall dwell 


And they did humble themselves even in the depths humility they did cry mightily God yea even all the day long did they cry unto their God that he would deliver them out their afflictions

And now the Lord was slow hear their cry because their iniquities nevertheless the Lord did hear their cries began soften the hearts the Lamanites that they began ease their burdens yet the Lord did not see fit deliver them out bondage

And it came pass that they began prosper by degrees in the land began raise grain more abundantly flocks herds that they did not suffer with hunger

Now there was great number women more than there was men therefore king Limhi commanded that every man should impart the support the widows their children that they might not perish with hunger this they did because the greatness their number that had been slain

Now the people Limhi kept together in body as much as it was possible secured their grain their flocks

And the king himself did not trust his person without th

And they began assemble themselves together unto the house the king And there came multitude their astonishment they beheld the king the queen their servants prostrate upon the earth they all lay there as though they were dead they also saw Ammon behold he was Nephite

And now the people began murmur among themselves some saying that it was great evil that had come upon them or upon the king his house because he had suffered that the Nephite should remain in the land

But others rebuked them saying The king hath brought this evil upon his house because he slew his servants who had had their flocks scattered at the waters Sebus

And they were also rebuked by those men who had stood at the waters Sebus scattered the flocks which belonged the king for they were angry with Ammon because the number which he had slain their brethren at the waters Sebus while defending the flocks the king

Now one them whose brother had been slain with the sword Ammon being exceedingly angry with Ammon drew h

And we see that these promises have been verified the people Nephi for it has been their quarrelings their contentions yea their murderings their plunderings their idolatry their whoredoms their abominations which were among themselves which brought upon them their wars their destructions

And those who were faithful in keeping the commandments the Lord were delivered at all times whilst thousands their wicked brethren have been consigned bondage or perish by the sword or dwindle in unbelief mingle with the Lamanites

But behold there never was happier time among the people Nephi since the days Nephi than in the days Moroni yea even at this time in the twenty first year the reign the judges

And it came pass that the twenty second year the reign the judges also ended in peace yea also the twenty third year

And it came pass that in the commencement the twenty fourth year the reign the judges there would also have been peace among the people Nephi had it not been for contention which to


Behold they do not desire that the Lord their God who hath created them should rule reign over them notwithstanding his great goodness his mercy towards them they do set at naught his counsels they will not that he should be their guide

O how great is the nothingness the children men yea even they are less than the dust the earth

For behold the dust the earth mov hither thither the dividing asunder at the command our great everlasting God

Yea behold at his voice do the hills the mountains tremble quake

And by the power his voice they are broken up become smooth yea even like unto valley

Yea by the power his voice doth the whole earth shake

Yea by the power his voice do the foundations rock even the very center

Yea if he say unto the earth Move it is moved

Yea if he say unto the earth Thou shalt go back that it lengthen out the day for many hours it is done

And thus according his word the earth go back it appear unto man that the sun stand still yea behold this is so for surel

And it came pass that the seventy first year passed away also the seventy second year yea in fine till the seventy ninth year had passed away yea even an hundred years had passed away the disciples Jesus whom he had chosen had all gone the paradise God save it were the three who should tarry there were other disciples ordained in their stead also many that generation had passed away

And it came pass that there was no contention in the land because the love God which did dwell in the hearts the people

And there were no envyings nor strifes nor tumults nor whoredoms nor lyings nor murders nor any manner lasciviousness surely there could not be happier people among all the people who had been created by the hand God

There were no robbers nor murderers neither were there Lamanites nor any manner ites but they were in one the children Christ heirs the kingdom God

And how blessed were they For the Lord did bless them in all their doings yea even they were blessed prospered until an hundr

In [9]:
# for chapter in range(len(bible['Genesis'])):
#     for verse in bible['Genesis'][chapter]:
#         words = verse.split(" ")
#         for word in words:
#             if word not in model:
#                 if not word.istitle():
#                     print(word, word in model)

unknown_words = {""}
for key in BoM.keys():
    for chapter in range(len(BoM[key])):
        for verse in BoM[key][chapter]:
            words = verse.split(" ")
            for word in words:
                if word not in model:
                    if not word.istitle():
                        unknown_words.add(word)
#                         print(word, word in model)

# unknown_words = {""}
# for key in BoM.keys():
#     for chapter in range(len(BoM[key])):
#         for verse in BoM[key][chapter]:
#             words = verse.split(" ")
#             for word in words:
#                 if word not in wiki_model:
#                     if not word.istitle():
#                         unknown_words.add(word)


print(unknown_words)
print(len(unknown_words))

{'', 'arrogancy', 'tortur', 'intrigues', 'verified', 'deadness', 'sett', 'temporally', 'desirest', 'overpowering', 'condescension', 'quaking', 'choirs', 'slothfulness', 'plunderers', 'repetitions', 'survived', 'forgiv', 'regained', 'frames', 'liberate', 'plunderings', 'senine', 'priestcraft', 'mete', 'idolatry', 'testifi', 'behoov', 'senums', 'disadvantages', 'ravenous', 'expertness', 'cavity', 'divides', 'concourses', 'visitations', 'scab', 'refiners', 'stomacher', 'dissenters', 'narrowness', 'respecter', 'metals', 'peaked', 'overbearance', 'priestcrafts', 'partaking', 'adulterous', 'imparting', 'spill', 'retaining', 'fortifications', 'agates', 'amnor', 'ignominious', 'wrongfully', 'enticed', 'girding', 'baseness', 'judg', 'pointers', 'reviled', 'bestead', 'satyrs', 'admonitions', 'awaken', 'imparted', 'cumbered', 'chasten', 'chastity', 'doubtings', 'consuming', 'fulfill', 'restoring', 'thieving', 'defending', 'delivering', 'obtaining', 'racked', 'prospering', 'mournings', 'burdened',

In [13]:
import gensim
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
# print('done')
# model.save('model')
print('done')

done


In [1]:
import multiprocessing
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.word2vec import Word2Vec
from gensim.models import word2vec
from nltk.corpus import gutenberg
from string import punctuation


sents = gutenberg.sents('bible-kjv.txt')
for file in gutenberg.fileids():
    if file != 'bible-kjv.txt':
        sents += gutenberg.sents(file)

# for sent in sents:
#     print(sent)

# print(len(bible_kjv_sents))
# print(len(sents))

discard_punctuation_and_lowercased_sents = [[word.lower() for word in sent if word not in punctuation] for sent in sents]
# # bible_model = word2vec.Word2Vec(discard_punctuation_and_lowercased_sents, min_count=5, size=300)

print('bible done')
wiki = WikiCorpus('data/sw/swwiki-latest-pages-articles.xml.bz2', 
                  lemmatize=False, dictionary={})
sentences = list(wiki.get_texts())
sents = sentences + discard_punctuation_and_lowercased_sents
print('len')
print(len(sents))
params = {'size': 300, 'window': 10, 'min_count': 10, 
          'workers': max(1, multiprocessing.cpu_count() - 1), 'sample': 1E-3,}
wiki_model = Word2Vec(sents, **params).wv
print('done')

bible done
len
120697
done
